## separate label and images after labelimg tool 

In [ ]:
import os
import shutil

In [ ]:
xml_list = [file for file in os.listdir('test/') if file.endswith('.xml')]
print(len(xml_list))
label_dir ='label/'
if not os.path.exists(label_dir):
        os.mkdir(label_dir)

In [ ]:
for i in xml_list:
    shutil.move('test/'+i, 'label/'+i)

#### convert  ICDAR format from VOC format

In [ ]:
from pathlib import Path
import xml.etree.ElementTree as ET
from typing import Dict
import os
import shutil

class XMLHandler:
    def __init__(self, xml_path: str or Path):
        self.xml_path = Path(xml_path)
        self.root = self.__open()

    def __open(self):
        with self.xml_path.open() as opened_xml_file:
            self.tree = ET.parse(opened_xml_file)
            return self.tree.getroot()

    def return_boxes_class_as_dict(self) -> Dict[int, Dict]:
        boxes_dict = {}
        for index, sg_box in enumerate(self.root.iter('object')):
            boxes_dict[index] = {"name": sg_box.find("name").text,
                                 "xmin": int(sg_box.find("bndbox").find("xmin").text),
                                 "ymin": int(sg_box.find("bndbox").find("ymin").text),
                                 "xmax": int(sg_box.find("bndbox").find("xmax").text),
                                 "ymax": int(sg_box.find("bndbox").find("ymax").text)}

        return boxes_dict


def converter(xml_files: str, output_folder: str) -> None:
    xml_files = [file for file in os.listdir('json_to_voc') if file.endswith('.xml')]

    for xml_index, xml in enumerate(xml_files):
        print(xml)
        file_name = xml[:-4]
        filename = "{}.txt".format(file_name)
        
        filename_path = Path(output_folder) / filename
        xml_content = XMLHandler('json_to_voc/'+xml)
        boxes = xml_content.return_boxes_class_as_dict()

        with open(filename_path, "a") as file:
            for box_index in boxes:
                box = boxes[box_index]
                if box['name']=='-':
                    continue
                print(box['name'])
                box_content = f"{box['name']} {box['xmin']} {box['ymin']} {box['xmax']} {box['ymax']}\n"
                file.write(box_content)

    print(f"Converted {len(xml_files)} files!")

    
if __name__ == '__main__':
    XML_FOLDER = "json_to_voc/"
    OUTPUT_FOLDER =  "icdar"
    if not os.path.exists('icdar/'):
        os.mkdir('icdar/')
    converter(XML_FOLDER, OUTPUT_FOLDER)

#### CRNN format changing

In [ ]:
file_list = [file for file in os.listdir('icdar') if file.endswith('.txt')]
print(len(file_list))
if not os.path.exists('images/'):
        os.mkdir('images/')

In [ ]:
n = 1
from PIL import Image
with open('train.txt','w') as f:
    for i in file_list:
        img_name = i[:-4]
        name = img_name+'.jpg'
        with open('icdar/'+i,'r') as a:
            lines = a.read().splitlines()
            for j in lines:
                line = j.split(' ')
                im = Image.open('JPEGImages/'+name)
                area = (int(line[1]), int(line[2]), int(line[3]), int(line[4]))
                cropped_im = im.crop(area)
                final_name = 'images/{}_{}.jpg'.format(line[0],str(n))
                cropped_im.save(final_name, quality=100, optimize=True)
                f.write('{}{}{}'.format(final_name,' ',line[0]))
                f.write('\n')
                n=n+1

### data augmentation for bus number detection

In [ ]:
import numpy as np
import os
import cv2

In [ ]:
def increase_brightness(img, value=10):
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    h, s, v = cv2.split(hsv)
    lim = 255 - value
    v[v > lim] = 255
    v[v <= lim] += value
    final_hsv = cv2.merge((h, s, v))
    img = cv2.cvtColor(final_hsv, cv2.COLOR_HSV2BGR)
    return img

In [ ]:
def noisy(noise_typ,image):
    if noise_typ == "gauss":
        row,col,ch= image.shape
        mean = 0
        var = 0.1
        sigma = var**0.5
        gauss = np.random.normal(mean,sigma,(row,col,ch))
        gauss = gauss.reshape(row,col,ch)
        noisy = image + gauss
        return noisy
    elif noise_typ == "s&p":
        row,col,ch = image.shape
        s_vs_p = 0.5
        amount = 0.004
        out = np.copy(image)
        # Salt mode
        num_salt = np.ceil(amount * image.size * s_vs_p)
        coords = [np.random.randint(0, i - 1, int(num_salt))for i in image.shape]
        out[coords] = 1
        # Pepper mode
        num_pepper = np.ceil(amount* image.size * (1. - s_vs_p))
        coords = [np.random.randint(0, i - 1, int(num_pepper))for i in image.shape]
        out[coords] = 0
        return out
    elif noise_typ == "poisson":
        vals = len(np.unique(image))
        vals = 2 ** np.ceil(np.log2(vals))
        noisy = np.random.poisson(image * vals) / float(vals)
        return noisy
    elif noise_typ =="speckle":
        row,col,ch = image.shape
        gauss = np.random.randn(row,col,ch)
        gauss = gauss.reshape(row,col,ch)        
        noisy = image + image * gauss
        return noisy

In [ ]:
def blur(src):
    dst = cv2.GaussianBlur(src,(3,3),cv2.BORDER_DEFAULT)
    return dst

In [ ]:
def contrast(img):
    alpha = 1.5 # Contrast control (1.0-3.0)
    beta = 0 # Brightness control (0-100)

    adjusted = cv2.convertScaleAbs(img, alpha=alpha, beta=beta)
    return adjusted

In [ ]:
def motion_horizontal_blur(img):
    size = randrange(1, 3)
    # generating the horizontal kernel
    kernel_motion_blur = np.zeros((size, size))
    kernel_motion_blur[int((size-1)/2), :] = np.ones(size)
    kernel_motion_blur = kernel_motion_blur / size

    # applying the kernel to the input image
    output = cv2.filter2D(img, -1, kernel_motion_blur)
    return output

In [ ]:
# def motion_vertical_blur(img):
#     size = randrange(1, 5)
#     kernel_motion_blur = np.zeros((size, size))
#     kernel_motion_blur[int(:, (size-1)/2)] = np.ones(size)
#     kernel_motion_blur = kernel_motion_blur / size
#     # applying the kernel to the input image
#     output = cv2.filter2D(img, -1, kernel_motion_blur)
#     return output

In [ ]:
if not os.path.exists('daimg/'):
    os.mkdir('daimg/')
if not os.path.exists('dalabel/'):
    os.mkdir('dalabel/')

In [ ]:
imagelist = [file for file in os.listdir('JPEGImages/') if file.endswith('.jpg')]
print(len(imagelist))

In [ ]:
import numpy as np
from random import randrange, uniform
num = 1
for img in imagelist:
    img_name = img[:-4]
    img = cv2.imread("JPEGImages/{}".format(img))
    txt_name = img_name+'.txt'
    # adding noise
    blur_img=True
    brightness =True
    noise = True
    #contrast_ratio =True
    #motion_blur_hori=True
    with open('dalabel/'+str(num)+'.txt','w') as df:
        with open('icdar/'+txt_name,'r') as nf:
            line = nf.read()
            df.write(line)
            cv2.imwrite("daimg/{}{}".format(num,'.jpg'),img)
            num=num+1
    if motion_blur_hori:
        motion_img = motion_horizontal_blur(img)
        with open('dalabel/'+str(num)+'_motion'+'.txt','w') as df:
            with open('icdar/'+txt_name,'r') as nf:
                line = nf.read()
                df.write(line)
                cv2.imwrite("daimg/{}_motion{}".format(num,'.jpg'),motion_img)
                num=num+1
#     if contrast_ratio:
#         cont_img =contrast(img)
#         with open('dalabel/'+str(num)+'_contrast'+'.txt','w') as df:
#             with open('icdar/'+txt_name,'r') as nf:
#                 line = nf.read()
#                 df.write(line)
#                 cv2.imwrite("daimg/{}_contrast{}".format(num,'.jpg'),cont_img)
#                 num=num+1
    if brightness:  
        bri = increase_brightness(img)     
        with open('dalabel/'+str(num)+'_brightness'+'.txt','w') as df:
            with open('icdar/'+txt_name,'r') as nf:
                line = nf.read()
                df.write(line)
                cv2.imwrite("daimg/{}_brightness{}".format(num,'.jpg'),bri)
                num=num+1
#     if blur_img:  
#         b_img = blur(img)     
#         with open('dalabel/'+str(num)+'_blur'+'.txt','w') as df:
#             with open('icdar/'+txt_name,'r') as nf:
#                 line = nf.read()
#                 df.write(line)
#                 cv2.imwrite("daimg/{}_blur{}".format(num,'.jpg'),b_img)
#                 num=num+1
            
    if noise:
        noi = ['s&p','gauss','s&p']
        n = np.random.choice(noi,1,replace = False)
        noise_sp = noisy(n,img)
        with open('dalabel/'+str(num)+'_noise'+'.txt','w') as df:
            with open('icdar/'+txt_name,'r') as nf:
                line = nf.read()
                df.write(line)
                cv2.imwrite("daimg/{}_noise{}".format(num,'.jpg'),noise_sp)
                num=num+1   

#### data augmentation for bus number recognition

In [ ]:
import numpy as np
import os
import cv2
import numpy as np
from random import randrange, uniform

In [ ]:
crop_images = 'images/'
imagelist = [file for file in os.listdir(crop_images) if file.endswith('.jpg')]
print(len(imagelist))

In [ ]:
def increase_brightness(img, value=30):
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    h, s, v = cv2.split(hsv)
    lim = 255 - value
    v[v > lim] = 255
    v[v <= lim] += value
    final_hsv = cv2.merge((h, s, v))
    img = cv2.cvtColor(final_hsv, cv2.COLOR_HSV2BGR)
    return img

In [ ]:
def motion_horizontal_blur(img):
    size = randrange(1, 3)
    # generating the horizontal kernel
    kernel_motion_blur = np.zeros((size, size))
    kernel_motion_blur[int((size-1)/2), :] = np.ones(size)
    kernel_motion_blur = kernel_motion_blur / size

    # applying the kernel to the input image
    output = cv2.filter2D(img, -1, kernel_motion_blur)
    return output

In [ ]:
def contrast(img):
    alpha = 1.5 # Contrast control (1.0-3.0)
    beta = 0 # Brightness control (0-100)

    adjusted = cv2.convertScaleAbs(img, alpha=alpha, beta=beta)
    return adjusted

In [ ]:
def noisy(noise_typ,image):
    if noise_typ == "gauss":
        row,col,ch= image.shape
        mean = 0
        var = 0.1
        sigma = var**0.5
        gauss = np.random.normal(mean,sigma,(row,col,ch))
        gauss = gauss.reshape(row,col,ch)
        noisy = image + gauss
        return noisy
    elif noise_typ == "s&p":
        row,col,ch = image.shape
        s_vs_p = 0.5
        amount = 0.004
        out = np.copy(image)
        # Salt mode
        num_salt = np.ceil(amount * image.size * s_vs_p)
        coords = [np.random.randint(0, i - 1, int(num_salt))for i in image.shape]
        out[coords] = 1
        # Pepper mode
        num_pepper = np.ceil(amount* image.size * (1. - s_vs_p))
        coords = [np.random.randint(0, i - 1, int(num_pepper))for i in image.shape]
        out[coords] = 0
        return out
    elif noise_typ == "poisson":
        vals = len(np.unique(image))
        vals = 2 ** np.ceil(np.log2(vals))
        noisy = np.random.poisson(image * vals) / float(vals)
        return noisy
    elif noise_typ =="speckle":
        row,col,ch = image.shape
        gauss = np.random.randn(row,col,ch)
        gauss = gauss.reshape(row,col,ch)        
        noisy = image + image * gauss
        return noisy

In [ ]:
def blur(src):
    dst = cv2.GaussianBlur(src,(5,5),cv2.BORDER_DEFAULT)
    return dst

In [ ]:
if not os.path.exists('R_daimg/'):
    os.mkdir('R_daimg/')

In [ ]:
num = 1
with open('train_da.txt','w') as df:
    with open('train.txt','r') as rd:
        abc = rd.readjson_list=[file for file in os.listdir('json/') if file.endswith('.json')]
print(len(json_list))
if not os.path.exists('json_to_voc'):
    os.makedirs('json_to_voc')().splitlines()       
        for img1 in abc:
            img = img1.split(' ')
            print(img[0])
            ab =img[1]
            img = cv2.imread("{}".format(img[0]))
            blur_img=True
            brightness =True
            noise = True
            contrast_ratio =True
            motion_blur_hori=True
            if motion_blur_hori:
                motion_img = motion_horizontal_blur(img)
                cv2.imwrite("R_daimg/{}{}".format(num,'.jpg'),motion_img)
                df.write("images/{}{}{}{}".format(num,'.jpg',' ',ab))
                df.write('\n')
                num=num+1
            if contrast_ratio:
                cont_img =contrast(img)
                cv2.imwrite("R_daimg/{}{}".format(num,'.jpg'),cont_img)
                df.write("images/{}{}{}{}".format(num,'.jpg',' ',ab))
                df.write('\n')
                num=num+1
            if brightness:  
                bri = increase_brightness(img)     
                cv2.imwrite("R_daimg/{}{}".format(num,'.jpg'),bri)
                df.write("images/{}{}{}{}".format(num,'.jpg',' ',ab))
                df.write('\n')
                num=num+1
            if blur_img:  
                b_img = blur(img)     
                cv2.imwrite("R_daimg/{}{}".format(num,'.jpg'),b_img)
                df.write("images/{}{}{}{}".format(num,'.jpg',' ',ab))
                df.write('\n')
                num=num+1

            if noise:
                noi = ['s&p','gauss','s&p']
                n = np.random.choice(noi,1,replace = False)
                noise_sp = noisy(n,img)
                cv2.imwrite("R_daimg/{}{}".format(num,'.jpg'),noise_sp)
                df.write("images/{}{}{}{}".format(num,'.jpg',' ',ab))
                df.write('\n')
                num=num+1   

### ICDar to YoLOv5 conversion

In [ ]:
import xml.etree.ElementTree as ET
import pickle
import os
import cv2
from os import listdir, getcwd
from os.path import join


def convert(size, box):
    dw = 1./size[0]
    dh = 1./size[1]
    x = (box[0] + box[1])/2.0
    y = (box[2] + box[3])/2.0
    w = box[1] - box[0]
    h = box[3] - box[2]
    x = x*dw
    w = w*dw
    y = y*dh
    h = h*dh
    return (round(x,5),round(y,5),round(w,5),round(h,5))

In [ ]:
aug_txt_folder = 'dalabel/'
aug_img_folder = 'daimg/'
Yolo_txt_folder = 'Y_dalabel/'
dalabel_list = [file for file in os.listdir(aug_txt_folder) if file.endswith('.txt')]
print(len(dalabel_list))

In [ ]:
if not os.path.exists(Yolo_txt_folder):
    os.mkdir(Yolo_txt_folder)

# single object detection

In [ ]:
# for i in dalabel_list:
#     y_img=cv2.imread(aug_img_folder+i[:-4]+'.jpg')
#     h, w = y_img.shape[:2]
#     in_file = open(aug_txt_folder+i,'r')
#     out_file = open(Yolo_txt_folder+i, 'w')
#     line = in_file.read().splitlines()
#     print(line)
#     for obj1 in line:
#         obj =obj1.split(' ')
#         b = (float(obj[1]), float(obj[3]), float(obj[2]), float(obj[4]))
#         bb = convert((w,h), b)
#         out_file.write(str(0) + " " + " ".join([str(a) for a in bb]) + '\n')

# multiple object detection

In [ ]:
for i in dalabel_list:
    y_img=cv2.imread(aug_img_folder+i[:-4]+'.jpg')
    h, w = y_img.shape[:2]
    in_file = open(aug_txt_folder+i,'r')
    out_file = open(Yolo_txt_folder+i, 'w')
    line = in_file.read().splitlines()
    string ='0123456789AB'
    str_list=list(string)
    #print(len(string))
    print(line)
    for obj1 in line:
        obj =obj1.split(' ')
        b = (float(obj[1]), float(obj[3]), float(obj[2]), float(obj[4]))
        bb = convert((w,h), b)
        str_val = str_list.index(str(obj[0]))
        print(str_val)
        out_file.write(str(str_val) + " " + " ".join([str(a) for a in bb]) + '\n')

## convert voc to yolov5

In [ ]:
import xml.etree.ElementTree as ET
import pickle
import os
from os import listdir, getcwd
from os.path import join

def convert(size, box):
    dw = 1./size[0]
    dh = 1./size[1]
    x = (box[0] + box[1])/2.0
    y = (box[2] + box[3])/2.0
    w = box[1] - box[0]
    h = box[3] - box[2]
    x = x*dw
    w = w*dw
    y = y*dh
    h = h*dh
    return (round(x,5),round(y,5),round(w,5),round(h,5))

def convert_annotation(image_id):
    in_file = open('label/%s.xml'%(image_id))
    out_file = open('labels/%s.txt'%( image_id), 'w')
    tree=ET.parse(in_file)
    root = tree.getroot()
    size = root.find('size')
    w = int(size.find('width').text)
    h = int(size.find('height').text)

    for obj in root.iter('object'):
        difficult = obj.find('difficult').text
        xmlbox = obj.find('bndbox')
        b = (float(xmlbox.find('xmin').text), float(xmlbox.find('xmax').text), float(xmlbox.find('ymin').text), float(xmlbox.find('ymax').text))
        bb = convert((w,h), b)
        out_file.write(str(0) + " " + " ".join([str(a) for a in bb]) + '\n')

In [ ]:
if not os.path.exists('labels/'):
    os.makedirs('labels/')
image_ids = [file for file in os.listdir('JPEGImages/') if file.endswith('.jpg')]
list_file = open('trainval.txt', 'w')
for image_ids in image_ids:
    image_id = image_ids[:-4]
    list_file.write('JPEGImages/%s.jpg\n'%(image_id))
    convert_annotation(image_id)
list_file.close()

##### some more things to add for boosting performance of object detection
#https://blog.gofynd.com/boost-object-detection-model-accuracy-552586d698c

In [ ]:
#JPEGImages/GPJ442_122.jpg

In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np


def unwarp(img, src, dst, testing):
    h, w = img.shape[:2]
    # use cv2.getPerspectiveTransform() to get M, the transform matrix, and Minv, the inverse
    M = cv2.getPerspectiveTransform(src, dst)
    # use cv2.warpPerspective() to warp your image to a top-down view
    warped = cv2.warpPerspective(img, M, (w, h), flags=cv2.INTER_LINEAR)

    if testing:
        f, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 10))
        f.subplots_adjust(hspace=.2, wspace=.05)
        ax1.imshow(img)
        x = [src[0][0], src[2][0], src[3][0], src[1][0], src[0][0]]
        y = [src[0][1], src[2][1], src[3][1], src[1][1], src[0][1]]
        ax1.plot(x, y, color='red', alpha=0.4, linewidth=3, solid_capstyle='round', zorder=2)
        ax1.set_ylim([h, 0])
        ax1.set_xlim([0, w])
        ax1.set_title('Original Image', fontsize=30)
        ax2.imshow(cv2.flip(warped, 1))
        ax2.set_title('Unwarped Image', fontsize=30)
        plt.show()
    else:
        return warped, M


im = cv2.imread("GPJ442_122.jpg")

w, h = im.shape[0], im.shape[1]
# We will first manually select the source points 
# we will select the destination point which will map the source points in
# original image to destination points in unwarped image
src = np.float32([(20,     1),
                  (540,  130),
                  (20,    520),
                  (570,  450)])

dst = np.float32([(600, 0),
                  (0, 0),
                  (600, 531),
                  (0, 531)])

unwarp(im, src, dst, True)


In [ ]:
import cv2
import math
import numpy as np
import matplotlib.pyplot as plt

img = cv2.imread('GPJ442_122.jpg')
rows,cols,ch = img.shape

pts1 = np.float32([[360,50],[2122,470],[2264, 1616],[328,1820]])

ratio=1.6
cardH=math.sqrt((pts1[2][0]-pts1[1][0])*(pts1[2][0]-pts1[1][0])+(pts1[2][1]-pts1[1][1])*(pts1[2][1]-pts1[1][1]))
cardW=ratio*cardH;
pts2 = np.float32([[pts1[0][0],pts1[0][1]], [pts1[0][0]+cardW, pts1[0][1]], [pts1[0][0]+cardW, pts1[0][1]+cardH], [pts1[0][0], pts1[0][1]+cardH]])

M = cv2.getPerspectiveTransform(pts1,pts2)

offsetSize=500
transformed = np.zeros((int(cardW+offsetSize), int(cardH+offsetSize)), dtype=np.uint8);
dst = cv2.warpPerspective(img, M, transformed.shape)

plt.subplot(121),plt.imshow(img),plt.title('Input')
plt.subplot(122),plt.imshow(dst),plt.title('Output')
plt.show()

# contrast

In [ ]:
import cv2
import numpy as np

# Open a typical 24 bit color image. For this kind of image there are
# 8 bits (0 to 255) per color channel
img = cv2.imread('mandrill.png')  # mandrill reference image from USC SIPI

s = 128
img = cv2.resize(img, (s,s), 0, 0, cv2.INTER_AREA)

def apply_brightness_contrast(input_img, brightness = 0, contrast = 0):
    
    if brightness != 0:
        if brightness > 0:
            shadow = brightness
            highlight = 255
        else:
            shadow = 0
            highlight = 255 + brightness
        alpha_b = (highlight - shadow)/255
        gamma_b = shadow
        
        buf = cv2.addWeighted(input_img, alpha_b, input_img, 0, gamma_b)
    else:
        buf = input_img.copy()
    
    if contrast != 0:
        f = 131*(contrast + 127)/(127*(131-contrast))
        alpha_c = f
        gamma_c = 127*(1-f)
        
        buf = cv2.addWeighted(buf, alpha_c, buf, 0, gamma_c)

    return buf

In [ ]:
import cv2

image = cv2.imread('GPJ442_122.jpg')

alpha = 1.5 # Contrast control (1.0-3.0)
beta = 0 # Brightness control (0-100)

adjusted = cv2.convertScaleAbs(image, alpha=alpha, beta=beta)

# cv2.imshow('original', image)
# cv2.imshow('adjusted', adjusted)
# cv2.waitKey()

In [ ]:
copyimport cv2
import numpy as np
image = cv2.imread("GPJ442_122.jpg")

alpha=1.5
beta=20

new_image=cv2.addWeighted(image,alpha,np.zeros(image.shape, image.dtype),0,beta)

# cv2.imshow("new",new_image)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

## apply vertical and horizental motion blur
#https://stackoverflow.com/questions/40305933/how-to-add-motion-blur-to-numpy-array

In [ ]:
import cv2
import numpy as np
from random import randrange, uniform

# randrange gives you an integral value
size = randrange(1, 5)
print(size)
img = cv2.imread('GPJ442_122.jpg')
cv2.imshow('Original', img)

# generating the kernel
kernel_motion_blur = np.zeros((size, size))
kernel_motion_blur[int((size-1)/2), :] = np.ones(size)
kernel_motion_blur = kernel_motion_blur / size

# applying the kernel to the input image
output = cv2.filter2D(img, -1, kernel_motion_blur)

cv2.imshow('Motion Blur', output)
cv2.waitKey(0)

In [ ]:
import cv2
import numpy as np

#size - in pixels, size of motion blur
#angel - in degrees, direction of motion blur
def apply_motion_blur(image, size, angle):
    k = np.zeros((size, size), dtype=np.float32)
    k[ (size-1)// 2 , :] = np.ones(size, dtype=np.float32)
    k = cv2.warpAffine(k, cv2.getRotationMatrix2D( (size / 2 -0.5 , size / 2 -0.5 ) , angle, 1.0), (size, size) )  
    k = k * ( 1.0 / np.sum(k) )        
    return cv2.filter2D(image, -1, k)

### voc data batch processing for changing label name

In [ ]:
import os
import xml.etree.ElementTree as ET

# : Batch modify the label name of the xml tag file in the VOC data set
def changelabelname(inputpath):
    listdir = os.listdir(inputpath)
    for file in listdir:
        if file.endswith('xml'):
            file = os.path.join(inputpath,file)
            tree = ET.parse(file)
            root = tree.getroot()
            for object1 in root.findall('object'):
                for sku in object1.findall('name'):           #Find the name you need to modify
                    if (sku.text == sku.text):               #‘preName’ is the name before the modification
                        sku.text = 'route'                 #‘TESTNAME’ is the modified name
                        tree.write(file,encoding='utf-8')     #Write into the original xml file and avoid the original xml Chinese characters garbled
                    else:
                        pass
        else:
            pass

if __name__ == '__main__':
    inputpath = 'label'  # 
    changelabelname(inputpath)

### icdar to voc format converter

In [ ]:
import os
import shutil
from os.path import join

import cv2
import numpy as np
import tqdm
from pascal_voc_writer import Writer
if not os.path.exists('voc_label'):
    os.makedirs('voc_label')


def convert_fn(txt_name):
    with open(f'ground_truth/{txt_name}') as f:
        lines = f.read().splitlines()
        xml_name=txt_name[:-4]
        img_name=txt_name[:-4]+'.jpg'
        path = 'test/'+img_name
        img = cv2.imread(path)
        h, w, _ = img.shape
        writer = Writer(img_name, w, h)
        for line in lines:
            lis= line.split(' ')
            writer.addObject(lis[0], lis[1], lis[2], lis[3], lis[4])
        writer.save(f'voc_label/{xml_name}.xml')
   


In [ ]:
txt_list = [file for file in os.listdir('ground_truth/') if file.endswith('.txt')]
print(len(txt_list))
for i in txt_list:
    convert_fn(i)

## Json format to voc format conversion

In [ ]:
import json
import os
from pascal_voc_writer import Writer
import cv2

In [ ]:
json_list=[file for file in os.listdir('json/') if file.endswith('.json')]
print(len(json_list))
if not os.path.exists('json_to_voc'):
    os.makedirs('json_to_voc')

In [ ]:
for i in json_list:
    img_name=i[:-5]+'.jpg'
    img_path='JPEGImages/'+img_name
    xml_name=i[:-5]+'.xml'
    img=cv2.imread(img_path)
    height, width, channels = img.shape
    writer= Writer(img_path,width,height)
    with open('json/'+i) as json_file:
        json_list=json.load(json_file)['shapes']
        for data in json_list:
            label=data['label']
            cords=data['points']
            xmin,ymin,xmax,ymax=int(cords[0][0]),int(cords[0][1]),int(cords[1][0]),int(cords[1][1])
            writer.addObject(label,xmin,ymin,xmax,ymax)
    writer.save('json_to_voc/'+xml_name)
            
    

# move validation data into different folder

In [ ]:
import shutil
if not os.path.exists('Val_JPEGimages'):
    os.makedirs('Val_JPEGimages')
if not os.path.exists('Val_icdar'):
    os.makedirs('Val_icdar')

In [ ]:
with open('val.txt') as f:
    lines =f.read().splitlines()
    for line in lines:
        li = line.split('/')
        ori_img_path=line
        img_name=li[1]
        icdar=li[1]
        icdar_name=icdar[:-4]+'.txt'
        icdar_path='Val_icdar/'+icdar_name
        ori_icdar_path='icdar/'+icdar_name
        img_path='Val_JPEGimages/'+img_name
        shutil.move(ori_img_path,img_path)
        shutil.move(ori_icdar_path,icdar_path)